In [1]:
import torch

In [2]:
tensor = torch.tensor([0.1, 0.2, 0.3])

In [3]:
X = tensor.reshape(1,3)

In [4]:
X

tensor([[0.1000, 0.2000, 0.3000]])

In [5]:
class Net(torch.nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(in_features=3, out_features=3, bias=True)
        self.fc2 = torch.nn.Linear(in_features=3, out_features=1, bias=True)
    
    def forward(self, x):
        print(x)
        print(x.shape)
        x = self.fc1(x)
        print(x)
        print(x.shape)
        x = self.fc2(x)
        print(x)
        print(x.shape)
        return x

In [6]:
model = Net()

In [7]:
par = []
for parameter in model.parameters():
    par.append(parameter)   

In [8]:
import torch
from torch.optim import Optimizer
class _RequiredParameter(object):
    """Singleton class representing a required parameter for an Optimizer."""
    def __repr__(self):
        return "<required parameter>"

required = _RequiredParameter()




class SGD(Optimizer):
    r"""Implements stochastic gradient descent (optionally with momentum).
    Nesterov momentum is based on the formula from
    `On the importance of initialization and momentum in deep learning`__.
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): learning rate
        momentum (float, optional): momentum factor (default: 0)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        dampening (float, optional): dampening for momentum (default: 0)
        nesterov (bool, optional): enables Nesterov momentum (default: False)
    Example:
        >>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    __ http://www.cs.toronto.edu/%7Ehinton/absps/momentum.pdf
    .. note::
        The implementation of SGD with Momentum/Nesterov subtly differs from
        Sutskever et. al. and implementations in some other frameworks.
        Considering the specific case of Momentum, the update can be written as
        .. math::
                  v = \rho * v + g \\
                  p = p - lr * v
        where p, g, v and :math:`\rho` denote the parameters, gradient,
        velocity, and momentum respectively.
        This is in contrast to Sutskever et. al. and
        other frameworks which employ an update of the form
        .. math::
             v = \rho * v + lr * g \\
             p = p - v
        The Nesterov version is analogously modified.
    """

    def __init__(self, params, lr=required, momentum=0, dampening=0,
                 weight_decay=0, nesterov=False):
        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))

        defaults = dict(lr=lr, momentum=momentum, dampening=dampening,
                        weight_decay=weight_decay, nesterov=nesterov)
        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")
        super(SGD, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(SGD, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('nesterov', False)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            dampening = group['dampening']
            nesterov = group['nesterov']

            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data)
                if momentum != 0:
                    param_state = self.state[p]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = torch.clone(d_p).detach()
                    else:
                        buf = param_state['momentum_buffer']
                        buf.mul_(momentum).add_(1 - dampening, d_p)
                    if nesterov:
                        d_p = d_p.add(momentum, buf)
                    else:
                        d_p = buf

                print('-'*15)
                print('Parameter before update:',p)
                print('Shape of parameter before update:',p.shape)
                print('Gradient:',d_p)
                print('Shape of the gradient:',d_p.shape)
                print('X:',X)
                print('Learning rate:',-group['lr'])
                check = p.clone()
                p.data.add_(-group['lr'], d_p)
                print('Parameter of the tensor after update:', p)
                print('Sanity check:', check +(-group['lr']) * d_p)
                print('-'*15)

        return loss

In [9]:
optimizer_detail = SGD(model.parameters(), lr=1)

In [14]:
criterion = torch.nn.MSELoss()


In [15]:
output = model(X)

tensor([[0.1000, 0.2000, 0.3000]])
torch.Size([1, 3])
tensor([[ 0.0003, -0.0732,  0.0420]], grad_fn=<AddmmBackward>)
torch.Size([1, 3])
tensor([[0.0361]], grad_fn=<AddmmBackward>)
torch.Size([1, 1])


In [16]:
# loss = output - 1

In [17]:
loss = criterion(1, output)

In [18]:
optimizer_detail.zero_grad()

In [19]:
loss.backward()

In [20]:
optimizer_detail.step()

---------------
Parameter before update: Parameter containing:
tensor([[-0.2920, -0.4200, -0.3449],
        [ 0.0455,  0.4253, -0.0052],
        [ 0.5471, -0.0541, -0.0723]], requires_grad=True)
Shape of parameter before update: torch.Size([3, 3])
Gradient: tensor([[-0.0405, -0.0810, -0.1215],
        [ 0.0795,  0.1591,  0.2386],
        [ 0.0164,  0.0327,  0.0491]])
Shape of the gradient: torch.Size([3, 3])
X: tensor([[0.1000, 0.2000, 0.3000]])
Learning rate: -1
Parameter of the tensor after update: Parameter containing:
tensor([[-0.2515, -0.3390, -0.2234],
        [-0.0340,  0.2662, -0.2438],
        [ 0.5308, -0.0868, -0.1214]], requires_grad=True)
Sanity check: tensor([[-0.2515, -0.3390, -0.2234],
        [-0.0340,  0.2662, -0.2438],
        [ 0.5308, -0.0868, -0.1214]], grad_fn=<AddBackward0>)
---------------
---------------
Parameter before update: Parameter containing:
tensor([ 0.2170, -0.1612,  0.0198], requires_grad=True)
Shape of parameter before update: torch.Size([3])
Gradi

# Calculations

First we have to take derivative of MSELoss.

In [21]:
(output-1)*2

tensor([[-1.9278]], grad_fn=<MulBackward0>)

```
---------------
Parameter before update: Parameter containing:
tensor([0.0094], requires_grad=True)
Shape of parameter before update: torch.Size([1])
Gradient: tensor([-1.9278])
Shape of the gradient: torch.Size([1])
X: tensor([[0.1000, 0.2000, 0.3000]])
Learning rate: -1
Parameter of the tensor after update: Parameter containing:
tensor([1.9372], requires_grad=True)
Sanity check: tensor([1.9372], grad_fn=<AddBackward0>)
---------------
```

We are dealing with optimization of  bias in the 2nd layer which is equal to wx+b w.r.t to b whtich is 1.

Using chain rule f'(loss(wx+b)) = f'(loss(wx+b)) * f'(wx+b) = [-1.9278] * 1 = [-1.9278]


In [33]:
(output-1)*2*1

tensor([[-1.9278]], grad_fn=<MulBackward0>)

We are dealing with optimization of weight in the 2nd layer which is equal to wx+b w.r.t to w whtich is x.


In [34]:
torch.tensor([-1.9278]) * torch.tensor([ 0.0003, -0.0732,  0.0420])

tensor([-0.0006,  0.1411, -0.0810])

We are dealing with optimization of bias in the 1st layer which is equal to wx+b w.r.t to b whtich is 1. And using chain rule we multiply derivative of loss function and derivative of 2nd layer wx+b w.r.t. x which w and 1.

In [35]:
torch.tensor([-1.9278]) * torch.tensor([ 0.2101, -0.4125, -0.0849])

tensor([-0.4050,  0.7952,  0.1637])

We are dealing with optimization of weight in the 1st layer which is equal to wx+b w.r.t to w whtich is x. And using chain rule we multiply derivative of loss function and derivative of 2nd layer wx+b w.r.t. x which w and x.

In [36]:
(torch.tensor([ 0.2101, -0.4125, -0.0849]) *
 torch.tensor([-1.9278])).reshape(3,1).matmul(torch.tensor([0.1000, 0.2000, 0.3000]).reshape(1,3))

tensor([[-0.0405, -0.0810, -0.1215],
        [ 0.0795,  0.1590,  0.2386],
        [ 0.0164,  0.0327,  0.0491]])